## 1. 問題の整理

* 与えられる: 整数 `a, b, x`
* 求める: 実数 `y = a^b` に「最も近い `x` の倍数」

  * 近さは絶対値距離 `|y - kx|` で定義
  * 距離が同じ倍数が複数ある場合は「数値として小さい方」を選ぶ
* 制約:

  * `1 ≤ x ≤ 10^9`
  * `1 ≤ a ≤ 10^9`
  * `-10^9 ≤ b ≤ 10^9`
  * `0 < a^b ≤ 10^9`（ここが重要）

サンプルより `4 -2 2` のケースで「`4^-2 = 1/16` なので 0 が最も近い倍数」となっていることから、`ab` は「積」ではなく「べき乗 `a^b`」です。

---

## 2. 数学的なポイント

### 2.1 最も近い倍数の一般形

`y = a^b`、`x > 0` とします。
`y` に最も近い `x` の倍数 `k x (k ∈ Z)` を求めたい。

実数 `y/x` に対して

* `q = floor(y / x)`
* 候補は `q * x` と `(q + 1) * x`
* 距離を比較すればよい:

```text
m1 = q * x
m2 = (q + 1) * x

d1 = |y - m1|
d2 = |m2 - y|
```

* `d1 < d2` → `m1`
* `d2 < d1` → `m2`
* `d1 == d2` → 小さい方なので `m1` を選ぶ

`y > 0` なので、負の倍数は 0 より左にあり、必ず距離が 0 より大きくなります。
したがって「0 を含む非負の倍数だけ」を見れば十分です。

### 2.2 負の指数 `b < 0` のとき

`b < 0` のとき `a^b = 1 / a^{-b}`。

* `a = 1` のとき

  * `1^b = 1`（b がいくつでも常に 1）
    → 普通に「1 に最も近い x の倍数」を求めればよい（整数問題）
* `a > 1` のとき

  * `a^b` は `0 < y ≤ 1` の範囲の分数
  * `x ≥ 1` なので、最初の非負倍数は `0` と `x`

    * `dist(0) = |y - 0| = y ≤ 1`
    * `dist(x) = |x - y| ≥ x - 1 ≥ 0`
  * `x ≥ 2` なら `x - 1 ≥ 1 > y` → 0 の方が必ず近い
  * `x = 1` のとき

    * `y ∈ (0,1]`
    * `dist(0) = y`、`dist(1) = |1 - y|`
    * ちょうど `y = 0.5` のときに同距離になるが「小さい方を選ぶ」ので 0

結論:

> `b < 0` かつ `a > 1` のとき、**答えは常に 0**。

よってこの場合は一切べき乗計算をせずに `0` を返してよい。

---

## 3. `b >= 0` のとき / `a == 1` のとき

* `b >= 0` かつ `a` 任意

  * `a` は整数なので `a^b` は整数
  * 制約 `0 < a^b ≤ 10^9` より、`a > 1` なら `b` は実質的にかなり小さい（`2^30 ≈ 10^9`）
  * Python の `pow(a, b)` は整数べき乗で `O(log b)` と高速、かつ誤差なし
* `a == 1`

  * `1^b` は b の符号に関わらず常に 1 なので、単に `y = 1` として扱う

したがって

* ケース1: `b < 0` かつ `a > 1` → `0` を即返す
* ケース2: 上以外（`b >= 0` or `a == 1`）→ 整数 `y = a^b` を求めて、整数演算で最も近い `x` の倍数を計算

---

## 4. アプローチ比較表（修正版）

| アプローチ | 概要                                      | 時間計算量 (1ケース) | 空間計算量 | Python実装コスト | 可読性 | CPythonとの相性       | 備考                               |
|-----------|-------------------------------------------|----------------------|------------|------------------|--------|------------------------|------------------------------------|
| 方法A     | `a**b` を float で計算し丸めて求める      | ほぼ O(1) ※内部は log| O(1)       | 低               | ★★★   | 浮動小数誤差の影響あり | 大きな a, b で誤差・オーバーフロー |
| 方法B     | 整数べき (`pow`) + 負指数は数学的に場合分け | O(log \|b\|)           | O(1)       | 中               | ★★★   | 整数演算のみで安定     | 制約に素直で再現性・安全性が高い   |

この問題では、精度と再現性・制約との整合性から **方法B** を採用します。

---

## 5. Python 特有の最適化と実装ポリシー

* `pow(a, b)` の整数べきは CPython が繰り返し二乗法で高速実装済み
* 全部整数演算なので浮動小数誤差ゼロ
* HackerRank の I/O テンプレートはそのまま使う（競技向けに十分高速）
* 型ヒントは `closestNumber` に付与し、Pylance で型エラーが出ないようにする
  （テンプレート側の未使用 import は HackerRank 由来なのでそのまま）

---

## 6. HackerRank 用完成コード

要求通り、「HackerRank での回答フォーマット」（指定された `closestNumber` 関数を実装）で提示します。

```python
#!/bin/python3

import math
import os
import random
import re
import sys


# Complete the 'closestNumber' function below.
#
# The function is expected to return an INTEGER.
# The function accepts following parameters:
#  1. INTEGER a
#  2. INTEGER b
#  3. INTEGER x
#
def closestNumber(a: int, b: int, x: int) -> int:
    # ケース1: b < 0 かつ a > 1
    #   y = a^b は 0 < y <= 1 の分数になり、
    #   x >= 1 の任意の正の倍数よりも 0 の方が必ず近い。
    if b < 0 and a > 1:
        return 0

    # ケース2: 上以外
    # a == 1 の場合は 1^b == 1 （b の符号に関わらず）
    if a == 1:
        y: int = 1
    else:
        # ここに入るのは
        #   - b >= 0 かつ a >= 2
        # の場合のみ（b < 0 かつ a > 1 はすでに return 済み）
        if b >= 0:
            y = pow(a, b)
        else:
            # 理論上ここに来るのは a == 1 の負指数のみだが、
            # 分岐構造を明確にするために冗長に書いておく。
            y = 1

    # y はここで必ず 1 以上の整数 (1 <= y <= 1e9)
    # y に最も近い x の倍数を整数演算で求める。
    q: int = y // x  # floor(y / x)
    m1: int = q * x
    m2: int = (q + 1) * x

    d1: int = abs(y - m1)
    d2: int = abs(m2 - y)

    if d1 < d2:
        return m1
    if d2 < d1:
        return m2
    # 距離が同じ場合は「より小さい方」を返すルールなので m1 を返す
    return m1


if __name__ == '__main__':
    fptr = open(os.environ['OUTPUT_PATH'], 'w')

    t = int(input().strip())

    for t_itr in range(t):
        first_multiple_input = input().rstrip().split()

        a = int(first_multiple_input[0])
        b = int(first_multiple_input[1])
        x = int(first_multiple_input[2])

        result = closestNumber(a, b, x)

        fptr.write(str(result) + '\n')

    fptr.close()
```

### サンプルでの確認

* `a = 349, b = 1, x = 4`

  * `y = 349`
  * `q = 349 // 4 = 87`
  * `m1 = 348, m2 = 352`
  * `d1 = 1, d2 = 3` → `348`
* `a = 395, b = 1, x = 7`

  * `y = 395`, `q = 56`, `m1 = 392, m2 = 399`
  * `d1 = 3, d2 = 4` → `392`
* `a = 4, b = -2, x = 2`

  * `b < 0` かつ `a > 1` なので即 `0`

いずれもサンプル出力と一致します。

もし HackerRank 上での実行結果（Runtime / Memory の数字）が出たら、そのプロファイルを見ながら更に突っ込んだチューニングもできますが、この問題の制約であればこの実装で十分に上位帯の性能になります。


# Closest Number - a^b に最も近い x の倍数を求める

## Table of Contents

* [概要](#overview)
* [アルゴリズム要点（TL;DR）](#tldr)
* [図解](#figures)
* [正しさのスケッチ](#correctness)
* [計算量](#complexity)
* [Python 実装](#impl)
* [CPython 最適化ポイント](#cpython)
* [エッジケースと検証観点](#edgecases)
* [FAQ](#faq)

---

<h2 id="overview">概要</h2>

### 問題要約

* 与えられる値: 整数 `a, b, x`
* 定義する値: 実数 `y = a^b`
* 目的: `y` に最も近い **`x` の整数倍** `k * x (k ∈ Z)` を求める
* 距離の定義: 絶対値距離 `|y - kx|`
* **同じ距離の倍数が複数ある場合は、数値として小さい方を選ぶ**

制約（HackerRank 問題より）:

* テストケース数 `1 ≤ T ≤ 10^5`
* `1 ≤ x ≤ 10^9`
* `1 ≤ a ≤ 10^9`
* `-10^9 ≤ b ≤ 10^9`
* `0 < a^b ≤ 10^9`（ここが重要）
* 出力: 各テストケースごとに 1 行、条件を満たす `x` の倍数（整数）

この制約により、

* `a^b` は **常に正**（0 より大きく 10^9 以下）
* `a` は正、`b` は正・0・負を取りうる
* `b` が負のとき `a^b` は 0 と 1 の間の分数になる可能性がある

---

<h2 id="tldr">アルゴリズム要点（TL;DR）</h2>

* 目標値を `y = a^b` とし、`y` に最も近い `x` の倍数を求める問題
* **負の指数の特別扱い**:

  * `b < 0` かつ `a > 1` のとき `0 < y ≤ 1`
  * このとき任意の正の倍数 `x, 2x, ...` よりも **常に 0 の方が近い**
    → この条件では **無条件で答えは `0`**
* それ以外（`b ≥ 0` または `a == 1`）のとき:

  * `a^b` は整数 (または `1^b = 1`)
  * Python の `pow(a, b)` で高精度な整数べき計算（誤差なし）
  * `q = y // x` として

    * 候補1 `m1 = q * x`
    * 候補2 `m2 = (q + 1) * x`
    * 距離 `|y - m1|` と `|m2 - y|` を比較
    * 距離が等しければ **小さい方 `m1`** を採用
* 時間計算量（1ケースあたり）: `O(log |b|)`（整数べき演算）
* 空間計算量: `O(1)`（定数個の整数のみ）

---

<h2 id="figures">図解</h2>

### フローチャート（メインロジック）

```mermaid
flowchart TD
  Start[Start] 
  NegCheck[Check sign of b</br> and value of a] 
  ReturnZero[Return 0] 
  ComputeY[Compute y</br> as a^b] 
  ComputeQ[Compute q as floor</br> y divided by x] 
  Cand[Compute candidates</br> m1 and m2] 
  Compare[Compare distances</br> and tie break] 
  Ret[Return result]

  Start --> NegCheck
  NegCheck --> ReturnZero
  NegCheck --> ComputeY
  ComputeY --> ComputeQ
  ComputeQ --> Cand
  Cand --> Compare
  Compare --> Ret
```

* `NegCheck` で「`b < 0` かつ `a > 1`」を検査し、該当時は `ReturnZero` に直行
* それ以外の場合は `ComputeY` で整数べき `y = a^b`（または `1`）を計算
* `ComputeQ` で `q = y // x` を求め、`Cand` で `m1 = q * x, m2 = (q + 1) * x` を生成
* `Compare` で距離とタイブレークを判定し、`Ret` で結果を返す

---

### データフロー図

```mermaid
graph LR
  subgraph InputStage
    In[Input a b x] --> Norm[Apply negative</br> exponent rule]
  end
  subgraph Core
    Norm --> Pow[Integer pow a^b</br> or constant 1]
    Pow --> Nearest[Nearest</br> multiple computation]
  end
  Nearest --> Out[Output result]
```

* `InputStage` では入力に対して「負の指数で 0 が確定するケース」を早期判定
* `Core` では

  * `Pow` で整数べき（または `1`）を計算
  * `Nearest` で最も近い `x` の倍数を選択
* `Out` が最終的な整数出力

---

<h2 id="correctness">正しさのスケッチ</h2>

### 1. 負の指数（`b < 0`）かつ `a > 1` の場合

このとき

* `a > 1` より `a^{-b} ≥ a^1 > 1`
* `b < 0` より `y = a^b = 1 / a^{-b}` は
  `0 < y ≤ 1` の範囲の分数

`x ≥ 1` なので、`x` の非負倍数は

* `0, x, 2x, ...`

と並ぶ。`y` は `0` と `x` の間にある（もしくは `x = 1` のときは `[0,1]` の内部）。

距離を比較すると:

* `dist(0) = |y - 0| = y ≤ 1`
* `dist(x) = |x - y| ≥ x - 1`

  * `x ≥ 2` のとき `x - 1 ≥ 1 > y` なので `dist(0) < dist(x)`
  * `x = 1` のとき `y ∈ (0,1]`

    * `dist(0) = y`
    * `dist(1) = |1 - y|`
    * `y = 0.5` のときのみ両距離が等しいが、ルールで「小さい方」を選ぶため 0

したがって、この条件では **常に 0 が最も近い倍数** であり、実装の早期 return `0` は正しい。

---

### 2. `b ≥ 0` または `a == 1` の場合

この場合、

* `a` は整数
* `b ≥ 0` なら `a^b` は整数
* `a == 1` なら `1^b` は b の符号に関わらず常に `1`

したがって `y` を整数として扱ってよい（`1 ≤ y ≤ 10^9`）。

整数 `y` に対して `x > 0` の最も近い倍数は、実数 `y / x` を考えると

* `q = floor(y / x)` とすると `q ≤ y/x < q + 1`
* `q` と `q + 1` の間に `y/x` が存在するので、最も近い整数は

  * `q` または `q + 1` のいずれか
* 距離は

  * `|y/x - q| = (y - qx) / x`
  * `|y/x - (q + 1)| = ((q + 1)x - y) / x`
* よって `y` に最も近い倍数は

  * `m1 = q * x`
  * `m2 = (q + 1) * x`

  のいずれかになる。

距離が等しい場合、問題は「数値として小さい方」を要求するので `m1` を返せばよい。
実装では `d1 < d2`, `d2 < d1` を順に判定し、それ以外（`d1 == d2`）を `m1` にフォールバックしており、要件を満たす。

---

### 3. 網羅性と終了性

* 入力 `(a, b, x)` に対して分岐は以下のみ:

  1. `b < 0` かつ `a > 1` → 0 を返す
  2. それ以外 → 整数べき計算と倍数探索
* いずれの経路でも有限回の整数演算（`pow` の内部は繰り返し二乗法）で終了するため、必ず停止する。
* 制約 `0 < a^b ≤ 10^9` により、`pow(a, b)` が巨大な整数になってメモリを圧迫することもない。

---

<h2 id="complexity">計算量</h2>

* 1 テストケースあたり

  * 整数べき計算 `pow(a, b)` のコスト: `O(log |b|)`
  * 後続の除算・乗算・絶対値・比較: すべて `O(1)`

* よって時間計算量（1 ケース）:

  * **Time: `O(log |b|)`**

* 空間計算量:

  * 補助変数は整数定数個 (`y, q, m1, m2, d1, d2`)
  * 配列や追加のデータ構造を持たない

  → **Space: `O(1)`**

テストケース数 `T ≤ 10^5` でも

* `O(T log |b|)` であり、`|b|` が大きく見えても `a^b ≤ 10^9` 制約のため実質的には小さめに抑えられる（大きな `b` と大きな `a` の組み合わせは存在しない）。

---

<h2 id="impl">Python 実装</h2>

* 実行環境: Python 3.11+ / CPython
* 形式: LeetCode 風の `class Solution` 実装
* 関数シグネチャ:

  ```python
  class Solution:
      def closestNumber(self, a: int, b: int, x: int) -> int:
          ...
  ```

### 実装コード

```python
from __future__ import annotations


class Solution:
    def closestNumber(self, a: int, b: int, x: int) -> int:
        """
        a^b に最も近い x の倍数を返す。

        仕様:
        - a, b, x は整数
        - 0 < a^b <= 1e9 が保証される
        - 複数の倍数が同じ距離の場合は、数値として小さい方を返す
        """

        # 負の指数かつ a > 1 の場合は、a^b は 0 と 1 の間の分数になる。
        # 任意の正の倍数 x, 2x, ... よりも 0 の方が常に近いため、
        # この条件では無条件で 0 を返してよい。
        if b < 0 and a > 1:
            return 0

        # ここから先は a^b を整数として扱えるケース。
        # a == 1 のときは 1^b == 1 （b の符号に依らない）。
        if a == 1:
            y: int = 1
        else:
            if b >= 0:
                # 整数同士のべき乗なので pow は高精度な整数演算。
                y = pow(a, b)
            else:
                # 構造上ここに来るのは a == 1 かつ b < 0 の場合のみ。
                # それでも 1^b は常に 1 なので安全。
                y = 1

        # y はここで必ず 1 <= y <= 1e9 の整数。
        # y に最も近い x の倍数は q = floor(y / x) 周辺の 2 点のみを見ればよい。
        q: int = y // x  # floor(y / x)

        m1: int = q * x          # 候補1: 下側の倍数
        m2: int = (q + 1) * x    # 候補2: 上側の倍数

        # 絶対値距離を比較し、より近い方を選ぶ。
        # 距離が等しい場合は、より小さい方（m1）を返す。
        d1: int = abs(y - m1)
        d2: int = abs(m2 - y)

        if d1 < d2:
            return m1
        if d2 < d1:
            return m2
        return m1
```

---

<h2 id="cpython">CPython最適化ポイント</h2>

* **整数べき演算 `pow(a, b)` の活用**

  * CPython の `pow(int, int)` は繰り返し二乗法ベースで実装されており、`O(log |b|)` で高速
  * 浮動小数点 (`float`) を一切使わないため、誤差や丸めによる不安定性がない
* **整数演算のみで完結**

  * `//`, `*`, `abs`, 比較演算のみでロジックが完結しており、JIT なしでも十分高速
* **分岐による早期リターン**

  * 負の指数で 0 が確定するケースを早期に返すことで、無駄なべき乗計算を省略
* **追加データ構造なし**

  * リスト・辞書等を一切持たず、レジスタ上の整数演算のみなのでキャッシュ局所性が良い
* **例外・型チェックの削減**

  * 問題側で制約が保証されている前提で実装しており、余計なバリデーションを省いている
    （競技プログラミング前提の最適化）

---

<h2 id="edgecases">エッジケースと検証観点</h2>

以下のケースをテストしておくと実装の妥当性が確認しやすいです。

1. **負の指数で 0 になるケース**

   * `a = 4, b = -2, x = 2` → `4^-2 = 1/16` に最も近い 2 の倍数は 0
   * `a = 2, b = -1, x = 5` → `1/2` に最も近い 5 の倍数も 0

2. **`a == 1` のケース**

   * `a = 1, b = 10, x = 3` → `1^10 = 1` に最も近い 3 の倍数は 0 または 3、距離:

     * `dist(0) = 1`, `dist(3) = 2` → 0
   * `a = 1, b = -100, x = 2` → `1^-100 = 1` として扱い、やはり 0 が最も近い 2 の倍数

3. **`b == 0` のケース**

   * `a = 5, b = 0, x = 4` → `5^0 = 1` と上記と同様
   * 任意の `a >= 1` で `a^0 = 1` になることを確認しておく

4. **ちょうど中間のタイブレーク**

   * `y = 6, x = 4` のような状況 (`a, b` を調整して作る)

     * 倍数は ..., 4, 8, ...
     * 距離: `|6 - 4| = 2`, `|8 - 6| = 2` で同距離
     * 問題仕様より、**小さい方の 4 を返す**必要がある
       → 実装では `d1 == d2` の場合に `m1` を返すことで満たしている

5. **大きなべき乗境界**

   * 制約ギリギリを意識したケース（実際には問題側が保証）

     * 例: `a = 2, b = 30, x = 7`（`2^30 = 1073741824` は制約を少し超えるが、考え方の検証用）
     * 実際のテストでは `0 < a^b ≤ 10^9` を満たす組み合わせだけが出る前提

6. **x が 1 のケース**

   * `x = 1` のときは「最も近い整数」を返す問題そのものになる
   * 本実装は `q = y // 1 = y` となり

     * `m1 = y`
     * `m2 = y + 1`
     * 明らかに `m1` の方が近い（距離 0）ので、常に `y` を返す

---

<h2 id="faq">FAQ</h2>

**Q1. なぜ浮動小数点 `a**b` を使わないのですか？**</br>
A1. 制約上 `a^b ≤ 10^9` とはいえ、`a, b` の取り得る範囲は広く、浮動小数点で `a**b` を計算すると精度劣化や丸め誤差で「最も近い倍数」がズレるリスクがあります。整数べき `pow(a, b)` + 場合分けで、**誤差ゼロ・再現性あり**の実装にできます。

---

**Q2. なぜ `b < 0` で `a > 1` のときは必ず 0 が答えになるのですか？**</br>
A2. この条件で `y = a^b` は必ず `(0,1]` の範囲に入る分数になります。一方、`x ≥ 1` の最小の正の倍数は `x` です。`0` と `x` の距離を比較すると、常に `0` の方が近い（または同距離のときも 0 を優先）ため、負の指数かつ `a > 1` のときは **どんな `x` でも答えは 0** になります。

---

**Q3. `a == 1` で `b < 0` のときの扱いは？**</br>
A3. `1^b` は `b` が負でも常に `1` です。したがって `y = 1` として、一般の整数ケースと同様に「1 に最も近い x の倍数」を求めれば十分です。本実装では `a == 1` のとき一括して `y = 1` としています。

---

**Q4. タイブレークで「小さい方」をどうやって保証していますか？**</br>
A4. 候補 `m1`（下側の倍数）と `m2`（上側の倍数）について `d1 < d2`, `d2 < d1` を順に判定し、どちらでもない場合 (`d1 == d2`) は `m1` を返す実装になっています。`m1 < m2` であることから、「距離が等しいときは小さい方」を確実に選択できます。

---

**Q5. 計算量的に `T = 10^5` でも安全ですか？**</br>
A5. 1 ケースあたり `O(log |b|)` ですが、問題側が `0 < a^b ≤ 10^9` を保証しているため、実際に出題される `a, b` の組み合わせで `|b|` が極端に大きくなることはありません。CPython の整数べき演算は非常に高速で、`T = 10^5` でも現実的な制限時間内に十分収まります。
